In [85]:
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np

### A/B тестирование

В этом задании вы познакомитесь с A/B тестированием и примените полученные знания по статистике. 

Рассмотрим A/B тестирование на примере сайта. У сайта есть два дизайна - старый и новый, и мы хотим оценить, насколько новый дизайн лучше старого. Для этого пользователи сайта случайным образом разделяются на контрольную и тестовую группы. Контрольной группе показывается старая версия сайта, тестовой группе - измененная версия. Оценить изменение можно несколькими способами, самый простой - оценить конверсию. Конверсия - доля пользователей, совершивших заранее определенное действие(например подписка, нажатие на кнопку, заполнение формы).

### Описание данных

Для начала нужно загрузить данные из файла `a_b_testing.csv` при помощи функции `read_csv` из библиотеки `pandas`. В данном случае 1 - была совершена подписка на сайт, 0 - подписки не было. A - контрольная группа, B - тестовая группа.

Далее нужно выполнить следующие пункты, описание выходного формата содержится внутри каждого задания.

### Доверительный интервал

В видео про доверительный интервал мы рассмотрели, как можно аппроксимировать биномиальное распределение нормальным. В некоторых случаях параметры нормального распределения можно вывести математически и ниже мы рассмотрим как.

Представим количество пользователей как случайную величину из биномиального распределения с параметрами  `n`  - количество пользователей и `p` - вероятность конверсии или как сумму  `n`  независимых бросаний монетки. Определим следующую случайную величину:

$$Y = X_{1} + X_{2} + \dots + X_{n} , \, $$
где случайная величина $X_{i}$ имеет распределение Бернулли. Для случайной величины $Y$ математическое ожидание и дисперсия равны:

$$\mu = np, \, \sigma^{2} = np\cdot(1 - p)$$

Далее применяя центральную предельную теорему(случайные величины $X_{i}$ распределены независимо и размер выборки большой), получаем что 

$$Y \sim \mathcal{N}(np \, np\cdot(1 - p))\ $$

Мы перешли от биномиального распределения к нормальному. Следующий шаг - стандартизация нормального распределения:

$$Z = \frac{Y - np}{\sqrt{np\cdot(1-p)}} \sim \mathcal{N}(0, \, 1) $$ 

Преобразуем выражение выше:

$$Z = \frac{Y - np}{\sqrt{np\cdot(1-p)}} = \frac{\frac{Y}{n} - p}{\sqrt{\frac{p(1-p)}{n}}} \sim \mathcal{N}(0, \, 1) $$

Так как среднее значение по выборке - это наблюдаемый процент конверсии, то доверительный интервал будет выглядеть следующим образом:

$${P}\left(p - z_{1-\frac{\alpha}{2}} \sqrt{\frac{p(1-p)}{n}} \le \mu \le p + z_{1-\frac{\alpha}{2}}\sqrt{\frac{p(1-p)}{n}}\right) = 1-\alpha$$

### ЗАДАНИЕ

Найдите доверительный интервал для среднего количества пользователей, совершивших подписку из контрольной выборки с уровнем значимости 95%.

#### РЕШЕНИЕ

In [86]:
ab_data = pd.read_csv('a_b_testing.csv')

In [87]:
ab_data.head()

,converted,group
0,1,A
1,0,A
2,0,A
3,1,B
4,1,A


In [88]:
def compute_ci(p, n):
    z_value = stats.norm.ppf(q=0.975)
    interval = z_value * (np.sqrt((p * (1 - p)/n)))
    conf_int = (p - interval, p + interval)
    return conf_int

In [89]:
a_group = ab_data.loc[ab_data['group'] == 'A', ['converted']]

In [90]:
n = a_group.shape[0]

In [91]:
n

2037

In [92]:
a_subscribed = a_group[a_group['converted'] == 1]

In [93]:
subsribed_count = a_subscribed.shape[0]

In [94]:
subsribed_count

839

In [95]:
p = subsribed_count / n

In [96]:
p

0.4118802160039273

In [97]:
interval = compute_ci(p, n)

In [98]:
interval

(0.3905069477313722, 0.43325348427648247)

In [99]:
n * interval[0]

795.4626525288052

In [100]:
n * interval[1]

882.5373474711948

In [101]:
answer1 = ", ".join(map(str, interval))

Запишите значения левой и правой границ через запятую, сохраняя приведенный порядок, в переменную `answer1`, которая будет являтся строкой

### Задача A/B тестирования

Рассмотрим независимые выборки $X$ и $Y$ для которых есть $\mu_x$ и $\mu_y$, определяющие среднее значение распределения.

Рассматривается следующая гипотеза:
$$
H_0: \mu_x = \mu_y
$$
против альтернативы:

$$
H_1: \mu_x \ne \mu_y.
$$

Если гипотеза $H_0$ отвергается, то показатель действительно поменялся.

Также можно тест можно записать и другим способом:
$$
H_0: \mu_x \le \mu_y
$$

против альтернативы:

$$
H_1: \mu_x > \mu_y
$$

### Статистика Стьюдента ЗАДАНИЕ

Найдите значение статистики Стьюдента в предположении независимости выборок

$$
T(X, Y) = \frac{\bar{X} - \bar{Y}}{\sqrt{\frac{s_x^2}{n} + \frac{s_y^2}{m}}}
$$

`n` - размер контрольной выборки, `m`  - размер тестовой выборки

Ответ запишите в переменную `answer2` с точностью до 2 знака после запятой

### РЕШЕНИЕ

### Статистика Стьюдента из библиотеки Scipy

Найдите p-value для статистики Стьюдента, используя функцию `stats.ttest_ind`.

### РЕШЕНИЕ

In [102]:
from scipy.stats import ttest_ind


In [104]:
b_group = ab_data.loc[ab_data['group'] == 'B', ['converted']]

In [109]:
stat_pvalue = ttest_ind(a_group, b_group)

In [111]:
stat_pvalue

Ttest_indResult(statistic=array([-1.6126205]), pvalue=array([0.10690591]))

In [113]:
answer2 = stat_pvalue[0][0]

In [114]:
answer2

-1.6126205013707797

In [116]:
answer3 = stat_pvalue[1][0]

In [117]:
answer3

0.10690590820177126

Дополнительная проверка: значение статистики Стьюдента, посчитанная двумя способами, должны совпадать

Ответ запишите в переменную `answer3` с точностью до 2 знака после запятой

### Ответы

In [118]:
output = """Confidence interval:{0}
T score custom {1:.2f}
p value {2:.2f}"""
print(output.format(answer1, answer2, answer3))

Confidence interval:0.3905069477313722, 0.43325348427648247
T score custom -1.61
p value 0.11
